In [1]:
import sys
import os
import torch
import pandas as pd
from torch.utils.data import Subset
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [2]:
from IPython.display import clear_output
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from dataset import AccelLaughterDataset, Subsample
from system import System
from segmentation_system import SegmentationSystem
from lared_laughter.constants import dataset_path

In [3]:
def do_fold(train_ds, test_ds):
    # data loaders
    data_loader_train = torch.utils.data.DataLoader(
        train_ds, batch_size=100, shuffle=True, num_workers=10,
        collate_fn=None)
    data_loader_val = torch.utils.data.DataLoader(
        test_ds, batch_size=100, shuffle=False, num_workers=10,
        collate_fn=None)

    system = SegmentationSystem()
    trainer = pl.Trainer(
        callbacks=[EarlyStopping(monitor="val_loss", mode="min")],
        accelerator='gpu',
        log_every_n_steps=1,
        max_epochs=-1)
    trainer.fit(system, data_loader_train, data_loader_val)

    trainer.test(system, data_loader_val)
    return system.test_results

In [4]:
def do_run(dataset, metrics_name='binary'):
    seed = 22
    cv_splits = KFold(n_splits=10, random_state=seed, shuffle=True).split(range(len(ds)))

    outputs = torch.empty((len(ds),))
    for f, (train_idx, test_idx) in enumerate(cv_splits):
        # create datasets    
        train_ds = Subset(dataset, train_idx)
        test_ds = Subset(dataset, test_idx)

        do_fold(train_ds, test_ds)
        # outputs[test_idx] = fold_outputs['proba'].cpu()
        clear_output(wait=True)

    # labels = torch.Tensor(ds.get_all_labels())
    # run_metrics = get_metrics(outputs, labels, metrics_name)

    return outputs, {}

In [5]:
examples = pd.read_csv('../dataset/computational_examples.csv')
examples = examples[examples['condition'] == 'video']
accel_ds_path = os.path.join(dataset_path, 'accel_long.pkl')
annot_path = os.path.join(dataset_path, 'continuous.pkl')
ds = AccelLaughterDataset(examples, accel_ds_path, annot_path, transform=Subsample(60, 60))

loaded 672 examples
467 have accel
447 have annotations


In [6]:
outputs, metrics = do_run(ds, 'binary')

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type               | Params
---------------------------------------------
0 | model | SegmentationResnet | 516 K 
---------------------------------------------
516 K     Trainable params
0         Non-trainable params
516 K     Total params
2.067     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.6435605883598328
        test_loss           0.8657689127014654
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
